In [1]:

if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales)

source("functions.r")
source("rating_variables.r")
monthlist <- sprintf("%02d", 1:11)
y <- 2010

Loading required package: pacman



# Plan (enrollment and contract) data

In [2]:
plan.data <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
  arrange(contractid, planid, state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid, planid) %>%
  fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
  ungroup()

plan.data.dt <- as.data.table(plan.data)
setorder(plan.data.dt, contractid, planid, fips, year, month)

plan.year <- plan.data.dt[
, {
    nonmiss <- !is.na(enrollment)
    n <- sum(nonmiss)
    list(
      n_nonmiss = n,
      avg_enrollment = if (n>0) mean(enrollment[nonmiss]) else NA_real_,
      sd_enrollment  = if (n>1) sd(enrollment[nonmiss]) else NA_real_,
      min_enrollment = if (n>0) min(enrollment[nonmiss]) else NA_real_,
      max_enrollment = if (n>0) max(enrollment[nonmiss]) else NA_real_,
      first_enrollment = if (n>0) enrollment[which(nonmiss)[1]] else NA_real_,
      last_enrollment  = if (n>0) enrollment[tail(which(nonmiss), 1)] else NA_real_,
      state  = tail(state, 1),
      county = tail(county, 1),
      org_type = tail(org_type, 1),
      plan_type = tail(plan_type, 1),
      partd = tail(partd, 1),
      snp   = tail(snp, 1),
      eghp  = tail(eghp, 1),
      org_name = tail(org_name, 1),
      org_marketing_name = tail(org_marketing_name, 1),
      plan_name = tail(plan_name, 1),
      parent_org = tail(parent_org, 1),
      contract_date = tail(contract_date, 1)
    )
  },
by = .(contractid, planid, fips, year)
]

plan.data.year <- as_tibble(plan.year)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

# Service Area data

In [3]:
service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

service.year <- service.year %>%
  arrange(contractid, fips, state, county, month)

service.year <- service.year %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
  ungroup()

service.data.year <- service.year %>%
  group_by(contractid, fips, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    state     = last(state),
    county    = last(county),
    org_name  = last(org_name),
    org_type  = last(org_type),
    plan_type = last(plan_type),
    partial   = last(partial),
    eghp      = last(eghp),
    ssa       = last(ssa),
    notes     = last(notes),
    .groups = "drop"
  )

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

# Penetration data

In [4]:
ma.penetration <- map_dfr(monthlist, ~ load_month_pen(.x, y)) %>%
  arrange(state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup()

pen.year <- ma.penetration %>%
  group_by(fips, state, county, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    n_elig  = sum(!is.na(eligibles)),
    n_enrol = sum(!is.na(enrolled)),

    avg_eligibles   = ifelse(n_elig  > 0, mean(eligibles, na.rm = TRUE), NA_real_),
    sd_eligibles    = ifelse(n_elig  > 1,  sd(eligibles,  na.rm = TRUE), NA_real_),
    min_eligibles   = ifelse(n_elig  > 0, min(eligibles,  na.rm = TRUE), NA_real_),
    max_eligibles   = ifelse(n_elig  > 0, max(eligibles,  na.rm = TRUE), NA_real_),
    first_eligibles = ifelse(n_elig  > 0, first(na.omit(eligibles)),     NA_real_),
    last_eligibles  = ifelse(n_elig  > 0,  last(na.omit(eligibles)),     NA_real_),

    avg_enrolled    = ifelse(n_enrol > 0, mean(enrolled,   na.rm = TRUE), NA_real_),
    sd_enrolled     = ifelse(n_enrol > 1,  sd(enrolled,    na.rm = TRUE), NA_real_),
    min_enrolled    = ifelse(n_enrol > 0, min(enrolled,    na.rm = TRUE), NA_real_),
    max_enrolled    = ifelse(n_enrol > 0, max(enrolled,    na.rm = TRUE), NA_real_),
    first_enrolled  = ifelse(n_enrol > 0, first(na.omit(enrolled)),       NA_real_),
    last_enrolled   = ifelse(n_enrol > 0,  last(na.omit(enrolled)),       NA_real_),

    ssa = last(ssa),
    .groups = "drop"
  )

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


# Star ratings data

In [5]:
ma.path.a <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/2010/2010_Part_C_Report_Card_Master_Table_2009_11_30_domain.csv"
star.data.a <- read_csv(ma.path.a, skip = 4,
                        col_names = rating.vars.2010, na = c("", "NA", "*"),
                        show_col_types = FALSE, progress = FALSE) %>%
  mutate(across(-any_of(c("contractid","org_type","contract_name","org_marketing")),
                ~ parse_number(as.character(.))))

ma.path.b <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/2010/2010_Part_C_Report_Card_Master_Table_2009_11_30_summary.csv"
star.data.b <- read_csv(ma.path.b, skip = 2,
                        col_names = c("contractid","org_type","contract_name","org_marketing","partc_score"),
                        na = c("", "NA", "*"),
                        show_col_types = FALSE, progress = FALSE) %>%
  mutate(
    new_contract = ifelse(partc_score == "Plan too new to be measured", 1, 0),
    partc_score  = ifelse(new_contract == 1, NA_real_, parse_number(as.character(partc_score)))
  ) %>%
  select(contractid, new_contract, partc_score) %>%
  mutate(partcd_score = NA_real_)

star.data.year <- star.data.a %>%
  select(-contract_name, -org_type, -org_marketing) %>%
  left_join(star.data.b, by = c("contractid")) %>%
  mutate(year = 2010)

Warning message:
“There were 33 warnings in `mutate()`.
The first warning was:
ℹ In argument: `across(...)`.
Caused by warning:
! 364 parsing failures.
row col expected                              actual
  1  -- a number Plan not required to report measure
  2  -- a number Plan not required to report measure
  3  -- a number Plan not required to report measure
  4  -- a number Plan too new to be measured        
  5  -- a number Plan too new to be measured        
... ... ........ ...................................
See problems(...) for more details.
ℹ Run `dplyr::last_dplyr_warnings()` to see the 32 remaining warnings.”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `partc_score = ifelse(new_contract == 1, NA_real_,
  parse_number(as.character(partc_score)))`.
Caused by warning:
! 342 parsing failures.
row col expected                                     actual
  1  -- a number Not enough data to calculate summary score
  2  -- a number Not enough data to calcul

# Benchmark data

In [6]:
bench.data <- read_csv("../../ma-data/ma/benchmarks/ratebook2010/CountyRate2010.csv",
                       skip = 10,
                       col_names = c("ssa","state","county_name","aged_parta",
                                     "aged_partb","disabled_parta","disabled_partb",
                                     "esrd_ab","risk_ab"),
                       show_col_types = FALSE, progress = FALSE)

bench.data.year <- bench.data %>%
  select(ssa, aged_parta, aged_partb, risk_ab) %>%
  mutate(ssa = as.numeric(ssa),
         risk_star5 = NA_real_, risk_star45 = NA_real_, risk_star4 = NA_real_,
         risk_star35 = NA_real_, risk_star3 = NA_real_, risk_star25 = NA_real_,
         risk_bonus5 = NA_real_, risk_bonus35 = NA_real_, risk_bonus0 = NA_real_,
         year = 2010)

# Merge data

In [13]:
ma.2010 <- plan.data.year %>%
  inner_join(service.data.year %>% select(contractid, fips),
             by = c("contractid","fips")) %>%
  filter(!state %in% c("VI","PR","MP","GU","AS",""),
         snp == "No",
         (planid < 800 | planid >= 900),
         !is.na(planid), !is.na(fips)) %>%
  left_join(pen.year %>% ungroup() %>%
            rename(state_long = state, county_long = county) %>%
            mutate(state_long = str_to_lower(state_long)) %>%
            group_by(fips) %>% mutate(ncount = n()) %>% filter(ncount == 1),
            by = c("fips")) %>%
  left_join(star.data.year %>%
            select(-any_of(c("contract_name","org_type","org_marketing"))),
            by = c("contractid")) %>%
  mutate(
    Star_Rating = case_when(
      partd == "No" ~ partc_score,
      partd == "Yes" & is.na(partcd_score) ~ partc_score,
      partd == "Yes" & !is.na(partcd_score) ~ partcd_score
    )
  ) %>%
  left_join(bench.data.year %>% filter(!is.na(ssa)), by = c("ssa")) %>%
  select(-starts_with("year.")) %>%
  mutate(year = y) %>%
  mutate(
    ma_rate = case_when(
      year < 2012 ~ risk_ab,
      year >= 2012 & year < 2015 & Star_Rating == 5    ~ risk_star5,
      year >= 2012 & year < 2015 & Star_Rating == 4.5  ~ risk_star45,
      year >= 2012 & year < 2015 & Star_Rating == 4    ~ risk_star4,
      year >= 2012 & year < 2015 & Star_Rating == 3.5  ~ risk_star35,
      year >= 2012 & year < 2015 & Star_Rating == 3    ~ risk_star3,
      year >= 2012 & year < 2015 & Star_Rating < 3     ~ risk_star25,
      year >= 2012 & year < 2015 & is.na(Star_Rating)  ~ risk_star35,
      year >= 2015 & Star_Rating >= 4                   ~ risk_bonus5,
      year >= 2015 & Star_Rating < 4                    ~ risk_bonus0,
      year >= 2015 & is.na(Star_Rating)                 ~ risk_bonus35
    )
  )

write_csv(ma.2010, "../data/data-2010.csv")

In [14]:
names(ma.2010)

[1] "contractid"          "planid"              "fips"               
 [4] "n_nonmiss"           "avg_enrollment"      "sd_enrollment"      
 [7] "min_enrollment"      "max_enrollment"      "first_enrollment"   
[10] "last_enrollment"     "state"               "county"             
[13] "org_type"            "plan_type"           "partd"              
[16] "snp"                 "eghp"                "org_name"           
[19] "org_marketing_name"  "plan_name"           "parent_org"         
[22] "contract_date"       "state_long"          "county_long"        
[25] "n_elig"              "n_enrol"             "avg_eligibles"      
[28] "sd_eligibles"        "min_eligibles"       "max_eligibles"      
[31] "first_eligibles"     "last_eligibles"      "avg_enrolled"       
[34] "sd_enrolled"         "min_enrolled"        "max_enrolled"       
[37] "first_enrolled"      "last_enrolled"       "ssa"                
[40] "ncount"              "breastcancer_screen" "rectalcancer_screen"
[43] "cv_diab_cholscreen"  "glaucoma_test"       "monitoring"         
[46] "flu_vaccine"         "pn_vaccine"          "physical_health"    
[49] "mental_health"       "osteo_test"          "physical_monitor"   
[52] "primaryaccess"       "osteo_manage"        "diab_healthy"       
[55] "bloodpressure"       "ra_manage"           "copd_test"          
[58] "bladder"             "falling"             "nodelays"           
[61] "doctor_communicate"  "carequickly"         "customer_service"   
[64] "overallrating_care"  "overallrating_plan"  "complaints_plan"    
[67] "appeals_timely"      "appeals_review"      "leave_plan"         
[70] "audit_problems"      "hold_times"          "info_accuracy"      
[73] "ttyt_available"      "new_contract"        "partc_score"        
[76] "partcd_score"        "Star_Rating"         "aged_parta"         
[79] "aged_partb"          "risk_ab"             "risk_star5"         
[82] "risk_star45"         "risk_star4"          "risk_star35"        
[85] "risk_star3"          "risk_star25"         "risk_bonus5"        
[88] "risk_bonus35"        "risk_bonus0"         "year"               
[91] "ma_rate"